In [2]:
from google.colab import drive
drive.mount('/content/drive/')

#!cp "/content/drive/My Drive/Dissertation/preprocessing.py" .
#!cp -r "/content/drive/My Drive/Dissertation/gym_maze" .
!cp "/content/drive/My Drive/Dissertation/envs/continuous_complex_cartpole.py" .

Mounted at /content/drive/


In [3]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/hiro_cartpole_rerun/{name}" 

    torch.save({
      'controller': {
          'critic': model.critic.state_dict(),
          'actor': model.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/hiro_cartpole_rerun/{name}" 
    checkpoint = torch.load(path)

    model.critic.load_state_dict(checkpoint['controller']['critic'])
    model.critic_target = copy.deepcopy(model.critic)
    
    model.actor.load_state_dict(checkpoint['controller']['actor'])
    model.actor_target = copy.deepcopy(model.actor)

    model.eval()

In [4]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [5]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [6]:
from continuous_complex_cartpole import ContinuousCartPoleEnv 
env = NormalizedEnv(ContinuousCartPoleEnv())

***

In [7]:
def plot_durations(episode_durations, actions):
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    
    axs[0].plot(durations_t, durations.numpy())

    durations_t, durations = list(map(list, zip(*actions)))
    durations = torch.tensor(durations, dtype=torch.float)
    axs[1].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [8]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [9]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [10]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [11]:
class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, 128)
        self.l5 = nn.Linear(128, 128)
        self.l6 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [12]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        #Create replay buffer
        self.memory = ReplayMemory(2000000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 5000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup=True, decay_epsilon=True):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

In [15]:
import time
SAVE_OFFSET = 0

def train_model():
    global SAVE_OFFSET

    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = TD3(n_observations, n_actions).to(device)
    
    max_episode_length = 200
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 100
    num_episodes = 2000 # M
    episode_durations = []
    actions = [(0,0)]

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        episode_steps = 0
        done = False
        while not done:        
            # agent pick action ...
            action = agent.select_action(state, i_episode <= warmup)

            # env response with next_observation, reward, terminate_info
            action_i = action.detach().cpu().squeeze(0).numpy()

            observation, reward, done, info = env.step(action_i)
            steps += 1

            #actions.append((steps, action_i[0]))
                
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            episode_steps += 1 
                
            extrinsic_reward = torch.tensor([reward], device=device)
            
            overall_reward += reward
            
            next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            # agent observe and update policy
            agent.observe(state, action, next_state, extrinsic_reward, done)           
            state = next_state
            
            if i_episode > warmup:
                agent.update_policy()

        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, actions)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 200 == 0:
                print(f"Episode {i_episode}: {np.mean(dur[-100:])}")
            if np.mean(dur[-100:]) >= 300:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"td3_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent
    return None

In [15]:
i = 1

while i < 10:
    agent = train_model()

    if agent is not None:
        print(i)
        i += 1

Episode 200: 10.77
Episode 400: 10.04
Solved after 550 episodes!
1
Episode 200: 11.52
Episode 400: 14.47
Solved after 470 episodes!
2
Episode 200: 10.52
Episode 400: 9.55
Solved after 592 episodes!
3
Episode 200: 12.15
Episode 400: 9.67
Episode 600: 151.45
Solved after 632 episodes!
4
Episode 200: 10.01
Episode 400: 13.68
Solved after 506 episodes!
5
Episode 200: 12.92
Episode 400: 12.29
Solved after 497 episodes!
6
Episode 200: 11.95
Episode 400: 29.33
Episode 600: 265.42
Solved after 644 episodes!
7
Episode 200: 12.32
Episode 400: 28.18
Solved after 542 episodes!
8
Episode 200: 9.92
Episode 400: 9.57
Solved after 539 episodes!
9


In [17]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
def eval_model(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 200
    num_episodes = 100

    for noise in np.arange(0,0.31,0.03):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                state = state + state_max * torch.FloatTensor(state.shape).uniform_(-noise/2, noise/2).to(device)
                state = state.float()

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[noise].append(overall_reward / num_episodes)

In [ ]:
state_min = torch.from_numpy(env.observation_space.low).to(device)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_max

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_action(state, agent, eps, target, targetted):
    state = state.clone().detach().requires_grad_(True)

    # initial forward pass
    action = agent.actor(state)
    action = torch.clamp(action, -1., 1.)

    if not targetted:
        loss = F.mse_loss(action, target)
    else:
        loss = F.mse_loss(action, target if action > 0 else -target)
    agent.actor.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return agent.select_action(state_p, False, False)

def apply_fgsm(agent, episode_durations, targetted):
    TARGET_ACTION = torch.tensor([[1.0]], device=device, dtype=torch.float)

    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 100

    for eps in np.arange(0.0, 0.031, 0.0025):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                action = fgsm_action(state, agent, eps, TARGET_ACTION, targetted)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i)
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [18]:
episodes = {}
for l2norm in np.arange(0,0.31,0.03):
    episodes[l2norm] = []

fgsm_t = {}
fgsm_ut = {}
for eps in np.arange(0.0, 0.031, 0.0025):
    fgsm_t[eps] = []
    fgsm_ut[eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 10:
    agent = train_model()
    #agent = TD3(n_observations, n_actions).to(device)
    #load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_model(agent, episodes)
        #apply_fgsm(agent, fgsm_t, True)
        #apply_fgsm(agent, fgsm_ut, False)
        print(f"{i} noise: {episodes}")
        #print(f"{i} fgsm (t): {fgsm_t}")
        #print(f"{i} fgsm (ut): {fgsm_ut}")
        i += 1

print("---")
print(f"noise: {episodes}")
#print(f"fgsm (t): {fgsm_t}")
#print(f"fgsm (ut): {fgsm_ut}")

Episode 200: 10.96
Episode 400: 95.29
Episode 600: 264.47
Episode 800: 224.66
Episode 1000: 241.36
Episode 1200: 221.02
Episode 1400: 228.68
Episode 1600: 208.19
Episode 1800: 203.11
Episode 200: 10.65
Episode 400: 12.45
Episode 600: 254.14
Episode 800: 252.97
Episode 1000: 260.18
Episode 1200: 252.68
Episode 1400: 251.42
Episode 1600: 251.61
Episode 1800: 261.06
Episode 200: 10.87
Episode 400: 9.75
Episode 600: 199.73
Episode 800: 235.22
Episode 1000: 270.3
Episode 1200: 254.63
Episode 1400: 267.88
Episode 1600: 255.98
Episode 1800: 266.22
Episode 200: 10.36
Episode 400: 9.3
Episode 600: 284.15
Solved after 648 episodes!
0 noise: {0.0: [300.64], 0.03: [169.8], 0.06: [72.56], 0.09: [46.6], 0.12: [33.88], 0.15: [33.44], 0.18: [26.7], 0.21: [26.3], 0.24: [26.97], 0.27: [25.29], 0.3: [27.38]}
Episode 200: 11.1
Episode 400: 217.73
Episode 600: 252.56
Episode 800: 235.86
Episode 1000: 244.92
Episode 1200: 226.25
Episode 1400: 237.87
Episode 1600: 263.19
Episode 1800: 259.23
Episode 200: 10.